## Gain experience applying scikit learn to machine learning problems

### Importing required library

In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#machine learning
from sklearn import preprocessing
from sklearn.ensemble import AdaBoostClassifier
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn import config_context
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay

##  1. Loading the HR Attrition Data

In [93]:
import os
path = os.getcwd()
path = os.getcwd() + '/data/IBM-HR-Data-Employee-Attrition.csv'
# importing data for car sales
dataset = pd.read_csv(path)
dataset.head()


,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


### 1.1  Dropping irrevalent  columns

In [51]:
#Get list of columns in the dataset
dataset.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [52]:
#Dropping columns (intution)
columns = ['DailyRate', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'HourlyRate', 'MonthlyRate',
        'Over18', 'RelationshipSatisfaction', 'StandardHours']
dataset.drop(columns, inplace=True, axis=1)

In [85]:
#To get description of all columns
dataset.describe(include = 'all')

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,...,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
count,1470.000000,1470,1470,1470,1470.000000,1470.000000,1470.000000,1470,1470.000000,1470.000000,...,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000,1470.000000
unique,NaN,2,3,3,NaN,NaN,NaN,2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,No,Travel_Rarely,Research & Development,NaN,NaN,NaN,Male,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,1233,1043,961,NaN,NaN,NaN,882,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,36.923810,NaN,NaN,NaN,9.192517,2.912925,2.721769,NaN,2.729932,2.063946,...,15.209524,3.153741,0.793878,11.279592,2.799320,2.761224,7.008163,4.229252,2.187755,4.123129
std,9.135373,NaN,NaN,NaN,8.106864,1.024165,1.093082,NaN,0.711561,1.106940,...,3.659938,0.360824,0.852077,7.780782,1.289271,0.706476,6.126525,3.623137,3.222430,3.568136
min,18.000000,NaN,NaN,NaN,1.000000,1.000000,1.000000,NaN,1.000000,1.000000,...,11.000000,3.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,30.000000,NaN,NaN,NaN,2.000000,2.000000,2.000000,NaN,2.000000,1.000000,...,12.000000,3.000000,0.000000,6.000000,2.000000,2.000000,3.000000,2.000000,0.000000,2.000000
50%,36.000000,NaN,NaN,NaN,7.000000,3.000000,3.000000,NaN,3.000000,2.000000,...,14.000000,3.000000,1.000000,10.000000,3.000000,3.000000,5.000000,3.000000,1.000000,3.000000
75%,43.000000,NaN,NaN,NaN,14.000000,4.000000,4.000000,NaN,3.000000,3.000000,...,18.000000,3.000000,1.000000,15.000000,3.000000,3.000000,9.000000,7.000000,3.000000,7.000000


##  2. Data cleaning and EDA

 ### This phase is the most time consuming yet the most importat one. Here, we filter and   extract only the information that is needed for problem solving.
 
   * Checking Data for Any missing levels and drop those corresponding rows
   * Split the data into features and labels(usually 'x' & 'y').
   * Finding out numerical and categorical columns
   * Filling (also called imputing) or disregarding missing values
   * Converting non-numerical values to numerical values(also called feature encoding.
   

### 2.1 Checking Data for Any missing levels and drop those corresponding rows.

In [53]:
# Checking the missing Data
print("Data for Missing Labels")
dataset.isna().sum()
# In below example the value of Attrition is showing '0' . 
# So it is an ideal data set so no need to remove any rows 

Data for Missing Labels


Age                        0
Attrition                  0
BusinessTravel             0
Department                 0
DistanceFromHome           0
Education                  0
EnvironmentSatisfaction    0
Gender                     0
JobInvolvement             0
JobLevel                   0
JobRole                    0
JobSatisfaction            0
MaritalStatus              0
MonthlyIncome              0
NumCompaniesWorked         0
OverTime                   0
PercentSalaryHike          0
PerformanceRating          0
StockOptionLevel           0
TotalWorkingYears          0
TrainingTimesLastYear      0
WorkLifeBalance            0
YearsAtCompany             0
YearsInCurrentRole         0
YearsSinceLastPromotion    0
YearsWithCurrManager       0
dtype: int64

### 2.2 Split the data into features and labels(usually 'x' & 'y').

In [54]:

X = dataset.drop("Attrition", axis=1)
# Select only the dependent variable or level
y = dataset["Attrition"]
print(y.head())

0    Yes
1     No
2    Yes
3     No
4     No
Name: Attrition, dtype: object


### 2.3 Finding out Numerical and Categorical Columns

In [55]:
# Finding out Numerical Columns
num_cols = X.select_dtypes("number").columns
#Finding out Categorical Columns
cat_cols = X.select_dtypes("object").columns
print(np.array(num_cols))
cat_cols

['Age' 'DistanceFromHome' 'Education' 'EnvironmentSatisfaction'
 'JobInvolvement' 'JobLevel' 'JobSatisfaction' 'MonthlyIncome'
 'NumCompaniesWorked' 'PercentSalaryHike' 'PerformanceRating'
 'StockOptionLevel' 'TotalWorkingYears' 'TrainingTimesLastYear'
 'WorkLifeBalance' 'YearsAtCompany' 'YearsInCurrentRole'
 'YearsSinceLastPromotion' 'YearsWithCurrManager']


Index(['BusinessTravel', 'Department', 'Gender', 'JobRole', 'MaritalStatus',
       'OverTime'],
      dtype='object')

### 2.4 Filling (also called imputing) or disregarding missing values

In [71]:

from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# Fill Categorical Values with "Missing" and Numerical Value with mean if any 
categorical_imputer = SimpleImputer(strategy="constant", fill_value="missing")
num_imputer = SimpleImputer(strategy="mean")

# Define Columns

categorical_features = np.array(cat_cols)
num_features = np.array(num_cols)

# Create an imputer (Something that fills missing data's)
imputer = ColumnTransformer([
                    ("categorical_imputer", categorical_imputer,categorical_features),
                    ("num_imputer", num_imputer,num_features),
                                ])


one_hot = OneHotEncoder()
ct = ColumnTransformer([("one_hot",
                                  one_hot,
                                  categorical_features)],
                                  remainder="passthrough")





## 3. Spliting the Data into Traiining and Test Data

In [72]:
np.random.seed(42)
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X,
                                                   y,
                                                  test_size=0.2
                                                  )

In [58]:
# X Train
print('On X train: ')
print('X train dimensions: ', X_train.shape)
display(X_train.head())

# X test
print('\nOn X test: ')
print('X test dimensions: ', X_test.shape)
display(X_test.head())

On X train: 
X train dimensions:  (1176, 25)


,Age,BusinessTravel,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1097,24,Travel_Rarely,Research & Development,21,2,3,Male,2,1,Laboratory Technician,...,14,3,3,2,3,3,1,1,0,0
727,18,Non-Travel,Research & Development,5,2,2,Male,3,1,Research Scientist,...,15,3,0,0,2,3,0,0,0,0
254,29,Travel_Rarely,Sales,20,2,4,Male,3,2,Sales Executive,...,14,3,1,10,2,3,3,2,0,2
1175,39,Travel_Rarely,Research & Development,12,3,4,Male,3,2,Manufacturing Director,...,21,4,0,7,3,3,5,4,1,0
1341,31,Travel_Rarely,Research & Development,20,3,2,Male,3,2,Laboratory Technician,...,11,3,1,10,2,3,10,8,0,2



On X test: 
X test dimensions:  (294, 25)


,Age,BusinessTravel,Department,DistanceFromHome,Education,EnvironmentSatisfaction,Gender,JobInvolvement,JobLevel,JobRole,...,PercentSalaryHike,PerformanceRating,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1041,28,Travel_Rarely,Sales,5,3,4,Male,3,2,Sales Executive,...,18,3,0,6,4,3,5,4,1,3
184,53,Travel_Rarely,Research & Development,13,2,4,Female,4,2,Manufacturing Director,...,11,3,2,5,3,3,4,2,1,3
1222,24,Travel_Rarely,Human Resources,22,1,4,Male,1,1,Human Resources,...,11,3,1,1,2,3,1,0,0,0
67,45,Travel_Rarely,Research & Development,7,3,2,Male,3,3,Research Scientist,...,17,3,1,25,2,3,1,0,0,0
220,36,Travel_Rarely,Research & Development,5,2,4,Male,3,2,Laboratory Technician,...,16,3,0,16,3,4,13,11,3,7


In [73]:
# X Train
print('On y train: ')
print('y train dimensions: ', y_train.shape)
display(y_train.head())

# X test
print('\nOn y test: ')
print('y test dimensions: ', y_test.shape)
display(y_test.head())

On y train: 
y train dimensions:  (1176,)


1097    No
727     No
254     No
1175    No
1341    No
Name: Attrition, dtype: object


On y test: 
y test dimensions:  (294,)


1041     No
184      No
1222    Yes
67       No
220      No
Name: Attrition, dtype: object

## 4. Creating Pipeline
###We can now establish the training pipeline for our models. Since this is a process we would need to repeat several times, it's good to essentially functionalize the process so we do not need to re-write redundant code. Here, we can impute some values that were missing, and encode any categorical values. Note that these pipelines will change according to the model and methodology you choose - additionally, the pipelines will also change depending on the data types of the columns in your dataset.

In [91]:
# This pipeline apply operations in steps as described 
# Step-1 Applying column transformer i.e converting non numeric to numeric and fill in missing values
# Step-2 Applying a ML Model

p= Pipeline([
    ("col_transform_step",ct),
    ("applying_ml_model",RandomForestClassifier(n_estimators=10,
              criterion='entropy'))
    
])
p.fit(X_train,y_train)
p.predict(X_test)
p.score(X_test,y_test)

with config_context(display='diagram'):
    display(p)





Pipeline(steps=[('col_transform_step',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('one_hot', OneHotEncoder(),
                                                  array(['BusinessTravel', 'Department', 'Gender', 'JobRole',
       'MaritalStatus', 'OverTime'], dtype=object))])),
                ('applying_ml_model',
                 RandomForestClassifier(criterion='entropy', n_estimators=10))])

## 5. Fit in to the model

In [94]:
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier

y_pred = p.predict(X_test)

p.fit(X_train,y_train)
#p.transform(X_train)
# Predicting the output 
print("Train Result:")
print("------------")
print("Classification Report: \n {}\n".format(classification_report(
      y_train, p.predict(X_train))))
#print("p.predict(X_test)

print("Confusion Matrix: \n {}\n".format(confusion_matrix(
      y_train, p.predict(X_train))))
res = cross_val_score(p, X_train, y_train,
              cv=10, scoring='accuracy')

print("Average Accuracy: \t {0:.4f}".format(np.mean(res)))
print("Accuracy SD: \t\t {0:.4f}".format(np.std(res)))
print("----------------------------------------------------------")


print("Test Result:")
print("-----------")
print("Classification Report: \n {}\n".format(
      classification_report(y_test, p.predict(X_test))))

print("Confusion Matrix: \n {}\n".format(
      confusion_matrix(y_test, p.predict(X_test))))

print("accuracy score: {0:.4f}\n".format(
      accuracy_score(y_test, p.predict(X_test))))




# Comparing the Prediction to the Real Output
#p.score(X_test,y_test)



Train Result:
------------
Classification Report: 
               precision    recall  f1-score   support

          No       0.98      1.00      0.99       978
         Yes       1.00      0.88      0.94       198

    accuracy                           0.98      1176
   macro avg       0.99      0.94      0.96      1176
weighted avg       0.98      0.98      0.98      1176


Confusion Matrix: 
 [[978   0]
 [ 23 175]]

Average Accuracy: 	 0.8546
Accuracy SD: 		 0.0213
----------------------------------------------------------
Test Result:
-----------
Classification Report: 
               precision    recall  f1-score   support

          No       0.89      0.98      0.93       255
         Yes       0.57      0.21      0.30        39

    accuracy                           0.87       294
   macro avg       0.73      0.59      0.62       294
weighted avg       0.85      0.87      0.85       294


Confusion Matrix: 
 [[249   6]
 [ 31   8]]

accuracy score: 0.8741



## 6. Cross validation and Hyper Parameter Tuning

In [95]:
# Cross Validation
p.score(X_test,y_test)

0.8741496598639455